In [62]:
X=np.array([[1,0],[0,1]])
Y=np.array([[2,2],[2,2]])
Z=np.dot(X,Y)
Z

array([[2, 2],
       [2, 2]])

## Portfolio Exercise: Starbucks
<br>

<img src="https://opj.ca/wp-content/uploads/2018/02/New-Starbucks-Logo-1200x969.jpg" width="200" height="200">
<br>
<br>
 
#### Background Information

The dataset you will be provided in this portfolio exercise was originally used as a take-home assignment provided by Starbucks for their job candidates. The data for this exercise consists of about 120,000 data points split in a 2:1 ratio among training and test files. In the experiment simulated by the data, an advertising promotion was tested to see if it would bring more customers to purchase a specific product priced at $10. Since it costs the company 0.15 to send out each promotion, it would be best to limit that promotion only to those that are most receptive to the promotion. Each data point includes one column indicating whether or not an individual was sent a promotion for the product, and one column indicating whether or not that individual eventually purchased that product. Each individual also has seven additional features associated with them, which are provided abstractly as V1-V7.

#### Optimization Strategy

Your task is to use the training data to understand what patterns in V1-V7 to indicate that a promotion should be provided to a user. Specifically, your goal is to maximize the following metrics:

* **Incremental Response Rate (IRR)** 

IRR depicts how many more customers purchased the product with the promotion, as compared to if they didn't receive the promotion. Mathematically, it's the ratio of the number of purchasers in the promotion group to the total number of customers in the purchasers group (_treatment_) minus the ratio of the number of purchasers in the non-promotional group to the total number of customers in the non-promotional group (_control_).

$$ IRR = \frac{purch_{treat}}{cust_{treat}} - \frac{purch_{ctrl}}{cust_{ctrl}} $$


* **Net Incremental Revenue (NIR)**

NIR depicts how much is made (or lost) by sending out the promotion. Mathematically, this is 10 times the total number of purchasers that received the promotion minus 0.15 times the number of promotions sent out, minus 10 times the number of purchasers who were not given the promotion.

$$ NIR = (10\cdot purch_{treat} - 0.15 \cdot cust_{treat}) - 10 \cdot purch_{ctrl}$$

For a full description of what Starbucks provides to candidates see the [instructions available here](https://drive.google.com/open?id=18klca9Sef1Rs6q8DW4l7o349r8B70qXM).

Below you can find the training data provided.  Explore the data and different optimization strategies.

#### How To Test Your Strategy?

When you feel like you have an optimization strategy, complete the `promotion_strategy` function to pass to the `test_results` function.  
From past data, we know there are four possible outomes:

Table of actual promotion vs. predicted promotion customers:  

<table>
<tr><th></th><th colspan = '2'>Actual</th></tr>
<tr><th>Predicted</th><th>Yes</th><th>No</th></tr>
<tr><th>Yes</th><td>I</td><td>II</td></tr>
<tr><th>No</th><td>III</td><td>IV</td></tr>
</table>

The metrics are only being compared for the individuals we predict should obtain the promotion – that is, quadrants I and II.  Since the first set of individuals that receive the promotion (in the training set) receive it randomly, we can expect that quadrants I and II will have approximately equivalent participants.  

Comparing quadrant I to II then gives an idea of how well your promotion strategy will work in the future. 

Get started by reading in the data below.  See how each variable or combination of variables along with a promotion influences the chance of purchasing.  When you feel like you have a strategy for who should receive a promotion, test your strategy against the test dataset used in the final `test_results` function.

In [58]:
# load in packages
from itertools import combinations

from test_results import test_results, score
import numpy as np
import pandas as pd
import scipy as sp
import sklearn as sk

import matplotlib.pyplot as plt
import seaborn as sb
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
%matplotlib inline

# load in the data
train_data = pd.read_csv('./training.csv')
test_data = pd.read_csv('Test.csv')
train_data.head()

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
0,1,No,0,2,30.443518,-1.165083,1,1,3,2
1,3,No,0,3,32.159350,-0.645617,2,3,2,2
2,4,No,0,2,30.431659,0.133583,1,1,4,2
3,5,No,0,0,26.588914,-0.212728,2,1,4,2
4,8,Yes,0,3,28.044332,-0.385883,1,1,2,2


In [28]:
# Cells for you to work and document as necessary - 
# definitely feel free to add more cells as you need

In [29]:
y_reg = train_data['purchase']
del train_data['purchase']

In [30]:
folds = KFold(n_splits=5, shuffle=True, random_state=0)

In [48]:
importances = pd.DataFrame()
oof_reg_preds = np.zeros(train_data.shape[0])
sub_reg_preds = np.zeros(test_data.shape[0])
train_features = train_data.columns[-7:]
for fold_, (trn_, val_) in enumerate(folds.split(train_data)):
    trn_x, trn_y = train_data[train_features].iloc[trn_], y_reg.iloc[trn_]
    val_x, val_y = train_data[train_features].iloc[val_], y_reg.iloc[val_]
    reg = lgb.LGBMClassifier(
        learning_rate=0.001,
        n_estimators=10000,
    
    )
    reg.fit(
        trn_x, trn_y,
        eval_set=[(val_x, val_y)],
        early_stopping_rounds=1000,
        verbose=100
    )
    imp_df = pd.DataFrame()
    imp_df['feature'] = train_features
    imp_df['gain'] = reg.booster_.feature_importance(importance_type='gain')
    
    imp_df['fold'] = fold_ + 1
    importances = pd.concat([importances, imp_df], axis=0, sort=False)
    
    oof_reg_preds[val_] = reg.predict(val_x, num_iteration=reg.best_iteration_)
    _preds = reg.predict(test_data[train_features], num_iteration=reg.best_iteration_)
    sub_reg_preds += np.expm1(_preds) / 5

Training until validation scores don't improve for 1000 rounds.
[100]	valid_0's binary_logloss: 0.0630819
[200]	valid_0's binary_logloss: 0.0629431
[300]	valid_0's binary_logloss: 0.0628295
[400]	valid_0's binary_logloss: 0.0627284
[500]	valid_0's binary_logloss: 0.062644
[600]	valid_0's binary_logloss: 0.0625783
[700]	valid_0's binary_logloss: 0.0625234
[800]	valid_0's binary_logloss: 0.0624775
[900]	valid_0's binary_logloss: 0.0624452
[1000]	valid_0's binary_logloss: 0.0624311
[1100]	valid_0's binary_logloss: 0.0624237
[1200]	valid_0's binary_logloss: 0.062428
[1300]	valid_0's binary_logloss: 0.0624358
[1400]	valid_0's binary_logloss: 0.0624411
[1500]	valid_0's binary_logloss: 0.0624382
[1600]	valid_0's binary_logloss: 0.0624402
[1700]	valid_0's binary_logloss: 0.0624466
[1800]	valid_0's binary_logloss: 0.0624443
[1900]	valid_0's binary_logloss: 0.0624409
[2000]	valid_0's binary_logloss: 0.0624438
Early stopping, best iteration is:
[1095]	valid_0's binary_logloss: 0.0624236


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Training until validation scores don't improve for 1000 rounds.
[100]	valid_0's binary_logloss: 0.0635364
[200]	valid_0's binary_logloss: 0.0633593
[300]	valid_0's binary_logloss: 0.0632417
[400]	valid_0's binary_logloss: 0.0631558
[500]	valid_0's binary_logloss: 0.0630912
[600]	valid_0's binary_logloss: 0.0630442
[700]	valid_0's binary_logloss: 0.0629976
[800]	valid_0's binary_logloss: 0.0629576
[900]	valid_0's binary_logloss: 0.0629285
[1000]	valid_0's binary_logloss: 0.062905
[1100]	valid_0's binary_logloss: 0.0628852
[1200]	valid_0's binary_logloss: 0.0628848
[1300]	valid_0's binary_logloss: 0.0628831
[1400]	valid_0's binary_logloss: 0.0628922
[1500]	valid_0's binary_logloss: 0.0628797
[1600]	valid_0's binary_logloss: 0.0628752
[1700]	valid_0's binary_logloss: 0.0628673
[1800]	valid_0's binary_logloss: 0.0628691
[1900]	valid_0's binary_logloss: 0.0628716
[2000]	valid_0's binary_logloss: 0.0628771
[2100]	valid_0's binary_logloss: 0.0628834
[2200]	valid_0's binary_logloss: 0.0629019


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Training until validation scores don't improve for 1000 rounds.
[100]	valid_0's binary_logloss: 0.0633535
[200]	valid_0's binary_logloss: 0.063235
[300]	valid_0's binary_logloss: 0.0631174
[400]	valid_0's binary_logloss: 0.063042
[500]	valid_0's binary_logloss: 0.062972
[600]	valid_0's binary_logloss: 0.0629182
[700]	valid_0's binary_logloss: 0.0628812
[800]	valid_0's binary_logloss: 0.0628558
[900]	valid_0's binary_logloss: 0.062842
[1000]	valid_0's binary_logloss: 0.0628426
[1100]	valid_0's binary_logloss: 0.0628528
[1200]	valid_0's binary_logloss: 0.0628689
[1300]	valid_0's binary_logloss: 0.0628854
[1400]	valid_0's binary_logloss: 0.0629056
[1500]	valid_0's binary_logloss: 0.0629236
[1600]	valid_0's binary_logloss: 0.0629381
[1700]	valid_0's binary_logloss: 0.06295
[1800]	valid_0's binary_logloss: 0.0629662
[1900]	valid_0's binary_logloss: 0.0629893
Early stopping, best iteration is:
[955]	valid_0's binary_logloss: 0.0628375


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Training until validation scores don't improve for 1000 rounds.
[100]	valid_0's binary_logloss: 0.0725217
[200]	valid_0's binary_logloss: 0.0724616
[300]	valid_0's binary_logloss: 0.0724075
[400]	valid_0's binary_logloss: 0.072362
[500]	valid_0's binary_logloss: 0.0723356
[600]	valid_0's binary_logloss: 0.0723331
[700]	valid_0's binary_logloss: 0.0723322
[800]	valid_0's binary_logloss: 0.0723457
[900]	valid_0's binary_logloss: 0.0723567
[1000]	valid_0's binary_logloss: 0.0723765
[1100]	valid_0's binary_logloss: 0.0723741
[1200]	valid_0's binary_logloss: 0.0723795
[1300]	valid_0's binary_logloss: 0.0724021
[1400]	valid_0's binary_logloss: 0.0724331
[1500]	valid_0's binary_logloss: 0.0724448
[1600]	valid_0's binary_logloss: 0.072454
Early stopping, best iteration is:
[676]	valid_0's binary_logloss: 0.0723297


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Training until validation scores don't improve for 1000 rounds.
[100]	valid_0's binary_logloss: 0.068448
[200]	valid_0's binary_logloss: 0.0682716
[300]	valid_0's binary_logloss: 0.0681221
[400]	valid_0's binary_logloss: 0.0680098
[500]	valid_0's binary_logloss: 0.0679078
[600]	valid_0's binary_logloss: 0.0678172
[700]	valid_0's binary_logloss: 0.0677522
[800]	valid_0's binary_logloss: 0.0676998
[900]	valid_0's binary_logloss: 0.0676688
[1000]	valid_0's binary_logloss: 0.067646
[1100]	valid_0's binary_logloss: 0.0676295
[1200]	valid_0's binary_logloss: 0.0676233
[1300]	valid_0's binary_logloss: 0.0676163
[1400]	valid_0's binary_logloss: 0.0676181
[1500]	valid_0's binary_logloss: 0.0676178
[1600]	valid_0's binary_logloss: 0.0676149
[1700]	valid_0's binary_logloss: 0.0676126
[1800]	valid_0's binary_logloss: 0.0675854
[1900]	valid_0's binary_logloss: 0.0675524
[2000]	valid_0's binary_logloss: 0.0675472
[2100]	valid_0's binary_logloss: 0.0675669
[2200]	valid_0's binary_logloss: 0.0675838
[

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [51]:
np.sum(sub_reg_preds!=0)

0

In [42]:
def promotion_strategy(df):
    '''
    INPUT 
    df - a dataframe with *only* the columns V1 - V7 (same as train_data)

    OUTPUT
    promotion_df - np.array with the values
                   'Yes' or 'No' related to whether or not an 
                   individual should recieve a promotion 
                   should be the length of df.shape[0]
                
    Ex:
    INPUT: df
    
    V1	V2	  V3	V4	V5	V6	V7
    2	30	-1.1	1	1	3	2
    3	32	-0.6	2	3	2	2
    2	30	0.13	1	1	4	2
    
    OUTPUT: promotion
    
    array(['Yes', 'Yes', 'No'])
    indicating the first two users would recieve the promotion and 
    the last should not.
    '''
    
    
    
    return sub_reg_preds

In [43]:
# This will test your results, and provide you back some information 
# on how well your promotion_strategy will work in practice

test_results(promotion_strategy)

0
0
0
0
Nice job!  See how well your strategy worked on our test data below!

Your irr with this strategy is nan.

Your nir with this strategy is 0.00.
We came up with a model with an irr of 0.0188 and an nir of 189.45 on the test set.

 How did you do?


/Users/shihaowen/Documents/Udacity-DSND/Experimental Design & Recommandations/Starbucks Project/test_results.py:22: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score_df = test_data.iloc[np.where(promos == 'Yes')]
/Users/shihaowen/Documents/Udacity-DSND/Experimental Design & Recommandations/Starbucks Project/test_results.py:13: RuntimeWarning: invalid value encountered in long_scalars
  irr = n_treat_purch / n_treat - n_ctrl_purch / n_control


(nan, 0.0)